<a href="https://colab.research.google.com/github/SYBIOTE/CMEPCMGN/blob/main/comouflage_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#repositories

In [ ]:
!git clone https://github.com/akisatok/pySaliencyMap
!git clone https://github.com/SYBIOTE/camogen
!git clone https://github.com/matterport/Mask_RCNN

#installation

In [ ]:
pip install Mask_RCNN/

In [ ]:
pip install /content/camogen

# image path 



In [ ]:
IMAGE_PATH = "/content/Unknown.jpeg"

# RCNN and detection

In [ ]:
%tensorflow_version 1.x

In [ ]:

import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/content/Mask_RCNN/")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
# Import COCO config
sys.path.append(os.path.join(ROOT_DIR, "samples/coco/"))  # To find local version
import coco

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

In [ ]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
               'bus', 'train', 'truck', 'boat', 'traffic light',
               'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
               'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
               'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
               'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
               'kite', 'baseball bat', 'baseball glove', 'skateboard',
               'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
               'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
               'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
               'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
               'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
               'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
               'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
               'teddy bear', 'hair drier', 'toothbrush']


In [ ]:
image = skimage.io.imread(IMAGE_PATH)

# Run detection
results = model.detect([image], verbose=1)

# Visualize results
r = results[0]
visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
                            class_names, r['scores'])

# mask and filtering

In [ ]:
plt.imshow(image)
plt.title("original")
filtered = image.copy()
cropped = image.copy()

### Remove Single Object

In [ ]:
i = 0 # i is the id of one particuar mask 
mask = r["masks"][:,:,i]
for j in range(cropped.shape[2]):
    cropped[:,:,j] = image[:,:,j] * mask[:,:]
    filtered[:,:,j] = image[:,:,j] - cropped[:,:,j]
plt.imshow(cropped)
plt.title("cropped")

In [ ]:
plt.imshow(filtered)
plt.title("filtered")

### Remove Multi Object 

In [ ]:

mask = r["masks"][:,:,0]
for i in range(1,r["masks"].shape[2]):
    mask += r["masks"][:,:,i]
for j in range(filtered.shape[2]):
  filtered[:,:,j] = image[:,:,j] - image[:,:,j] * mask
plt.imshow(filtered)
plt.title("filtered")


# Color Extraction

## load image

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import cv2
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import SpectralClustering

In [ ]:
dominant_color = 5
hexadecimal_color_minibatch_kmean = []
hexadecimal_color_kmean = []
score_minibatch = []
score = []

In [ ]:
img = filtered

x_end , y_end = img.shape[0:2]
print(img.shape)

plt.imshow(img ,extent=[0,y_end,x_end,0])

In [ ]:
# img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
plt.imshow(img , extent=[0,y_end,x_end,0])

In [ ]:
original_pixel = img.shape
print(original_pixel)

In [ ]:
pixel = img.reshape((x_end*y_end , 3))
print(pixel.shape)

## K-means minibatch

In [ ]:
%%time
km = MiniBatchKMeans( n_clusters = dominant_color)
km.fit(pixel)

In [ ]:
minibatch_centers = km.cluster_centers_
# A uint8 data type contains all whole numbers from 0 to 255.
minibatch_centers = np.array(minibatch_centers , dtype='uint8')
avg = [0,0,0]
for x in minibatch_centers:
    if not (x[0]==x[1]==x[2]==0):
      avg += (x/len(minibatch_centers)-1)
print(avg)
for x in minibatch_centers:
    if (x[0]==x[1]==x[2]==0):
      minibatch_centers= avg


print(minibatch_centers)
hexadecimal_color_minibatch_kmean.clear()
for array in minibatch_centers:
  hexadecimal_color_minibatch_kmean.append( '#%02x%02x%02x' % (array[0], array[1], array[2]) )

print(hexadecimal_color_minibatch_kmean)
#Plotting these colors

i =1
plt.figure(0,figsize=(8,2))


color = []
for x in minibatch_centers:
    plt.subplot(1,5,i)
    plt.axis("off")
    i+=1
    color.append(x)
    a = np.zeros((100,100,3),dtype='uint8')
    a[:,:,:] = x
    plt.imshow(a)

plt.show()



In [ ]:
%%time
score_minibatch.clear()
for i in range(dominant_color):
  sum=0
  for j in km.labels_:
    if(i == j):
      sum += 1
  score_minibatch.append(sum/len(km.labels_))

print(score_minibatch)
print(hexadecimal_color_minibatch_kmean)


### forward

In [ ]:
centers = km.cluster_centers_
# A uint8 data type contains all whole numbers from 0 to 255.
centers = np.array(centers , dtype='uint8')
print("After : ",centers)

In [ ]:
for array in centers:
  hexadecimal_color_kmean.append( '#%02x%02x%02x' % (array[0], array[1], array[2]) )


In [ ]:
hexadecimal_color_kmean

In [ ]:

#Plotting these colors

i =1
plt.figure(0,figsize=(8,2))


color = []
for x in centers:
    plt.subplot(1,5,i)
    plt.axis("off")
    i+=1
    color.append(x)
    
    a = np.zeros((100,100,3),dtype='uint8')
    a[:,:,:] = x
    plt.imshow(a)
plt.show()



In [ ]:
%%time
score = []
for i in range(dominant_color):
  sum=0
  for j in km.labels_:
    if(i == j):
      sum += 1
  score.append(sum/len(km.labels_))

print(score)

In [ ]:
# segmenting the original image
new_pixel = np.zeros((x_end*y_end , 3))

color 

In [ ]:
km.labels_

In [ ]:
for i in range(new_pixel.shape[0]):
  new_pixel[i] = color[km.labels_[i]]

new_pixel = new_pixel.reshape(original_pixel)

In [ ]:
plt.imshow(img ,extent=[0,y_end,x_end,0])
plt.title("Before segmentation")
plt.show()

In [ ]:
new_pixel /= 255

In [ ]:
plt.imshow(new_pixel , extent=[0,y_end,x_end,0])
plt.title("After Segmentation")
plt.show()

In [ ]:
new_pixel

In [ ]:
# gives error avoid for now
from PIL import Image
im = Image.fromarray((new_pixel * 255).astype(np.uint8))
plt.imshow(im)

#Camogen

In [ ]:
yp = hexadecimal_color_minibatch_kmean
hexadecimal_color_minibatch_kmean = []
for i in yp:
   if i == '#dbe1d9':
     pass
   else:
     hexadecimal_color_minibatch_kmean.append(i)

In [ ]:
import camogen
import matplotlib.pyplot as plt
from camogen.generate import generate


parameters = {'width': 100, 'height': 100, 'polygon_size': 50, 'color_bleed': 1,'max_depth':8,
              'colors': hexadecimal_color_minibatch_kmean,
              'spots': {'amount': 200000, 'radius': {'min': 1, 'max': 10}, 'sampling_variation': 5},
              'pixelize': {'percentage': 1, 'sampling_variation': 50, 'density': {'x': 100, 'y': 100}}}
camo_pattern = generate(parameters)
plt.imshow(camo_pattern)
# camo_pattern.save('/content/camogen/images/genrated.png')

#pattern overlappping

In [ ]:
# a = np.array(filtered)
# b = np.array(camo_pattern)
# for c in range(a.shape[2]):
#   for j in range(a.shape[1]):
#     for r in range(a.shape[0]):
#       if r == 0:
#         a[r,j,c] = camo_pattern[r,j,c]



In [ ]:
import PIL
from PIL import Image
import requests
from io import BytesIO
from PIL import ImageFilter
from PIL import ImageEnhance
from IPython.display import display
import numpy as np


pil_filtered = Image.fromarray(filtered)
pil_camo = Image.fromarray(np.array(camo_pattern))

pil_filtered

In [ ]:
 pil_filtered.size , pil_camo.size

In [ ]:
width, height = pil_filtered.size
camo_w, camo_h = pil_camo.size
camo_w, camo_h = camo_w//2, camo_h//2
for j in range(height):
  for r in range(width):
    rgb_values = pil_filtered.getpixel((r,j))
    if 0 in rgb_values:
      '''
      l = r
      k = j
      if l > camo_h or k > camo_w:
        l = random.randint(0,camo_w)
        k = random.randint(0,camo_h)
        '''
      l = r%camo_w
      k = j%camo_h
      pil_filtered.putpixel( (r,j), pil_camo.getpixel((l,k)))
     
pil_filtered

# Effectiveness check

## detection test

In [ ]:
pix = np.array(pil_filtered)

In [ ]:
test = model.detect([pix], verbose=1)

In [ ]:
# Visualize results
t = test[0]
visualize.display_instances(pix, t['rois'], t['masks'], t['class_ids'], 
                            class_names, t['scores'])

## saliency map



In [ ]:
cd pySaliencyMap/                

In [ ]:
import cv2
import matplotlib.pyplot as plt
import pySaliencyMap

MIN_W = 352
MIN_H = 256
print(pix.shape)
multiplier = 1
stepsize = .1
while True:
    if (multiplier> MIN_W/pix.shape[1] and multiplier> MIN_H/pix.shape[0]):
      break;
    else:
        multiplier+=stepsize
print(multiplier)
req_w,req_h = int(pix.shape[1]*multiplier),int(pix.shape[0]*multiplier)
pix = cv2.resize(pix, dsize=(req_w,req_h), interpolation=cv2.INTER_CUBIC)
imgsize = pix.shape
img_width  = imgsize[1]
img_height = imgsize[0]
print(img_width,img_height)
sm = pySaliencyMap.pySaliencyMap(img_width, img_height)

# computation
saliency_map = sm.SMGetSM(pix)
binarized_map = sm.SMGetBinarizedSM(pix)
salient_region = sm.SMGetSalientRegion(pix)
# visualize

plt.figure(figsize=[18,12])
plt.subplot(2,2,1), plt.imshow(pix)
plt.title('Input image')
plt.subplot(2,2,2), plt.imshow(saliency_map, 'gray')
plt.title('Saliency map')
plt.subplot(2,2,3), plt.imshow(binarized_map)
plt.title('Binarilized saliency map')
plt.subplot(2,2,4), plt.imshow(cv2.cvtColor(salient_region, cv2.COLOR_BGR2RGB))
plt.title('Salient region')

plt.show()